In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

import random, os
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, classification_report

%matplotlib inline

# 데이터 불러오기

In [2]:
df = pd.read_csv("bank-direct-marketing-campaigns.csv")
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,2,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,1,999,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,yes


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  campaign        41188 non-null  int64  
 11  pdays           41188 non-null  int64  
 12  previous        41188 non-null  int64  
 13  poutcome        41188 non-null  object 
 14  emp.var.rate    41188 non-null  float64
 15  cons.price.idx  41188 non-null  float64
 16  cons.conf.idx   41188 non-null  float64
 17  euribor3m       41188 non-null 

# 데이터 전처리

In [4]:
# 중복 제거
print(df.shape)
df = df.drop_duplicates(keep='last')
print(df.shape)

(41188, 20)
(39404, 20)


In [5]:
df.describe()

,age,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed
count,39404.000000,39404.000000,39404.000000,39404.000000,39404.000000,39404.000000,39404.000000,39404.000000,39404.000000
mean,40.116105,2.618744,960.847097,0.178738,0.064067,93.577538,-40.499604,3.601243,5165.986481
std,10.460328,2.814780,190.869184,0.503172,1.577041,0.583820,4.644327,1.742337,72.763866
min,17.000000,1.000000,0.000000,0.000000,-3.400000,92.201000,-50.800000,0.634000,4963.600000
25%,32.000000,1.000000,999.000000,0.000000,-1.800000,93.075000,-42.700000,1.334000,5099.100000
50%,38.000000,2.000000,999.000000,0.000000,1.100000,93.798000,-41.800000,4.857000,5191.000000
75%,47.000000,3.000000,999.000000,0.000000,1.400000,93.994000,-36.400000,4.961000,5228.100000
max,98.000000,56.000000,999.000000,7.000000,1.400000,94.767000,-26.900000,5.045000,5228.100000


In [6]:
pd.crosstab(df["default"], df["y"])

y,no,yes
default,,
no,26979,4156
unknown,7824,442
yes,3,0


In [7]:
df = df.replace({'y':{'no':0, 'yes':1}})
df = df.replace({'default': {'yes': 'unknown'}})
df = df.replace({'default':{'no':0, 'unknown':1}})
df = df.replace({'pdays': {999: -1}})
# df = df.replace({'month': {'may':5, 'jun':6, 'jul':7, 'aug':8, 'oct':10, 
#                            'nov':11, 'dec':12, 'mar':3, 'apr':4, 'sep':9}})
df = df.replace({'contact':{'telephone':0, 'cellular':1}})
df

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,0,no,no,0,may,mon,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,57,services,married,high.school,1,no,no,0,may,mon,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,40,admin.,married,basic.6y,0,no,no,0,may,mon,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,56,services,married,high.school,0,no,yes,0,may,mon,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
5,45,services,married,basic.9y,1,no,no,0,may,mon,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,0,yes,no,1,nov,fri,1,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1
41184,46,blue-collar,married,professional.course,0,no,no,1,nov,fri,1,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41185,56,retired,married,university.degree,0,yes,no,1,nov,fri,2,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41186,44,technician,married,professional.course,0,no,no,1,nov,fri,1,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1


In [8]:
# 데이터 분석에 도움되지 않는 변수 삭제
df.drop('day_of_week', axis=1 ,inplace=True)

In [9]:
# 10대 별로 나누어 분석
df['age'] = df['age']//10
df

,age,job,marital,education,default,housing,loan,contact,month,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,5,housemaid,married,basic.4y,0,no,no,0,may,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
1,5,services,married,high.school,1,no,no,0,may,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
3,4,admin.,married,basic.6y,0,no,no,0,may,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
4,5,services,married,high.school,0,no,yes,0,may,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
5,4,services,married,basic.9y,1,no,no,0,may,1,-1,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,7,retired,married,professional.course,0,yes,no,1,nov,1,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1
41184,4,blue-collar,married,professional.course,0,no,no,1,nov,1,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41185,5,retired,married,university.degree,0,yes,no,1,nov,2,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,0
41186,4,technician,married,professional.course,0,no,no,1,nov,1,-1,0,nonexistent,-1.1,94.767,-50.8,1.028,4963.6,1


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39404 entries, 0 to 41187
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             39404 non-null  int64  
 1   job             39404 non-null  object 
 2   marital         39404 non-null  object 
 3   education       39404 non-null  object 
 4   default         39404 non-null  int64  
 5   housing         39404 non-null  object 
 6   loan            39404 non-null  object 
 7   contact         39404 non-null  int64  
 8   month           39404 non-null  object 
 9   campaign        39404 non-null  int64  
 10  pdays           39404 non-null  int64  
 11  previous        39404 non-null  int64  
 12  poutcome        39404 non-null  object 
 13  emp.var.rate    39404 non-null  float64
 14  cons.price.idx  39404 non-null  float64
 15  cons.conf.idx   39404 non-null  float64
 16  euribor3m       39404 non-null  float64
 17  nr.employed     39404 non-null 

In [11]:
# One-Hot Encoding
df = pd.get_dummies(df, drop_first = True)
df

,age,default,contact,campaign,pdays,previous,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,...,month_dec,month_jul,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_nonexistent,poutcome_success
0,5,0,0,1,-1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,0,0,0,1,0
1,5,1,0,1,-1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,0,0,0,1,0
3,4,0,0,1,-1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,0,0,0,1,0
4,5,0,0,1,-1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,0,0,0,1,0
5,4,1,0,1,-1,0,1.1,93.994,-36.4,4.857,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,7,0,1,1,-1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,1,0
41184,4,0,1,1,-1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,1,0
41185,5,0,1,2,-1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,1,0
41186,4,0,1,1,-1,0,-1.1,94.767,-50.8,1.028,...,0,0,0,0,0,1,0,0,1,0


# 랜덤 시드 고정

In [12]:
SEED = 42

def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(SEED)

# data 분리

In [13]:
X = df.drop('y', axis=1)
y = df['y']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(31523, 47) (31523,) (7881, 47) (7881,) (7881, 47) (7881,)


In [14]:
# over-sampling
print(pd.Series(y_train).value_counts())
smote = SMOTE(k_neighbors = 3, random_state=SEED)
X_train, y_train = smote.fit_resample(X_train, y_train)
pd.Series(y_train).value_counts()

0    27836
1     3687
Name: y, dtype: int64


0    27836
1    27836
Name: y, dtype: int64

# Model

## LogisticRegression

In [15]:
model_LR = LogisticRegression(C=0.001) # C : 규제 강도
model_LR.fit(X_train, y_train)
y_predLR = model_LR.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_LR.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predLR))
print(classification_report(y_val, y_predLR, target_names=['no', 'yes']))

Train data 
 [[23374  4462]
 [ 8973 18863]]
Validation data 
 [[5849 1121]
 [ 366  545]]
              precision    recall  f1-score   support

          no       0.94      0.84      0.89      6970
         yes       0.33      0.60      0.42       911

    accuracy                           0.81      7881
   macro avg       0.63      0.72      0.66      7881
weighted avg       0.87      0.81      0.83      7881



/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


## KNeighborsClassifier

In [16]:
model_KNN = KNeighborsClassifier()
model_KNN.fit(X_train, y_train)
y_predKNN = model_KNN.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_KNN.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predKNN))
print(classification_report(y_val, y_predKNN, target_names=['no', 'yes']))

Train data 
 [[23240  4596]
 [  842 26994]]
Validation data 
 [[5408 1562]
 [ 404  507]]
              precision    recall  f1-score   support

          no       0.93      0.78      0.85      6970
         yes       0.25      0.56      0.34       911

    accuracy                           0.75      7881
   macro avg       0.59      0.67      0.59      7881
weighted avg       0.85      0.75      0.79      7881



## DecisionTreeClassifier

In [17]:
model_TREE = DecisionTreeClassifier(criterion='entropy', max_depth=9, random_state=SEED)
model_TREE.fit(X_train, y_train)
y_predTREE = model_TREE.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_TREE.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predTREE))
print(classification_report(y_val, y_predTREE, target_names=['no', 'yes']))

Train data 
 [[24610  3226]
 [ 8689 19147]]
Validation data 
 [[6079  891]
 [ 440  471]]
              precision    recall  f1-score   support

          no       0.93      0.87      0.90      6970
         yes       0.35      0.52      0.41       911

    accuracy                           0.83      7881
   macro avg       0.64      0.69      0.66      7881
weighted avg       0.86      0.83      0.85      7881



## SVC

In [18]:
model_SVC = SVC(kernel='linear')
model_SVC.fit(X_train, y_train)
y_predSVC = model_SVC.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_TREE.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predSVC))
print(classification_report(y_val, y_predSVC, target_names=['no', 'yes']))

Train data 
 [[24610  3226]
 [ 8689 19147]]
Validation data 
 [[6221  749]
 [ 435  476]]
              precision    recall  f1-score   support

          no       0.93      0.89      0.91      6970
         yes       0.39      0.52      0.45       911

    accuracy                           0.85      7881
   macro avg       0.66      0.71      0.68      7881
weighted avg       0.87      0.85      0.86      7881



## MLPClassifier

In [19]:
model_MLP = MLPClassifier(hidden_layer_sizes = (100, 100), max_iter=200, activation='relu', alpha=3)
model_MLP.fit(X_train, y_train)
y_preMLP = model_MLP.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_MLP.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_preMLP))
print(classification_report(y_val, y_preMLP, target_names=['no', 'yes']))

Train data 
 [[24272  3564]
 [10627 17209]]
Validation data 
 [[6057  913]
 [ 402  509]]
              precision    recall  f1-score   support

          no       0.94      0.87      0.90      6970
         yes       0.36      0.56      0.44       911

    accuracy                           0.83      7881
   macro avg       0.65      0.71      0.67      7881
weighted avg       0.87      0.83      0.85      7881



## RandomForestClassifier

In [20]:
model_RF = RandomForestClassifier()
model_RF.fit(X_train, y_train)
y_predRF = model_RF.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_RF.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predRF))
print(classification_report(y_val, y_predRF, target_names=['no', 'yes']))

Train data 
 [[27448   388]
 [  245 27591]]
Validation data 
 [[6310  660]
 [ 542  369]]
              precision    recall  f1-score   support

          no       0.92      0.91      0.91      6970
         yes       0.36      0.41      0.38       911

    accuracy                           0.85      7881
   macro avg       0.64      0.66      0.65      7881
weighted avg       0.86      0.85      0.85      7881



## BaggingClassifier

In [21]:
model_BAG = BaggingClassifier(
    DecisionTreeClassifier(),
    n_estimators=600,
    max_samples=0.6,
    max_features=0.6, 
    bootstrap=True,
    n_jobs=-1 
)
model_BAG.fit(X_train, y_train)
y_predBAG = model_BAG.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_BAG.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predBAG))
print(classification_report(y_val, y_predBAG, target_names=['no', 'yes']))

Train data 
 [[27252   584]
 [ 1633 26203]]
Validation data 
 [[6457  513]
 [ 508  403]]
              precision    recall  f1-score   support

          no       0.93      0.93      0.93      6970
         yes       0.44      0.44      0.44       911

    accuracy                           0.87      7881
   macro avg       0.68      0.68      0.68      7881
weighted avg       0.87      0.87      0.87      7881



## VotingClassifier

In [22]:
model_VOTING = VotingClassifier(estimators=[('LR', model_LR),
                                      ('KNN', model_KNN)],
                                voting='hard')
model_VOTING.fit(X_train, y_train)
y_predVOT = model_VOTING.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_VOTING.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predVOT))
print(classification_report(y_val, y_predVOT, target_names=['no', 'yes']))

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Train data 
 [[25633  2203]
 [ 9074 18762]]
Validation data 
 [[6280  690]
 [ 467  444]]
              precision    recall  f1-score   support

          no       0.93      0.90      0.92      6970
         yes       0.39      0.49      0.43       911

    accuracy                           0.85      7881
   macro avg       0.66      0.69      0.67      7881
weighted avg       0.87      0.85      0.86      7881



## AdaBoostClassifier

In [23]:
model_AB = AdaBoostClassifier(n_estimators=10, random_state=42)
model_AB.fit(X_train, y_train)
y_predAB = model_AB.predict(X_val)

print("Train data","\n",confusion_matrix(y_train, model_AB.predict(X_train)))
print("Validation data","\n",confusion_matrix(y_val, y_predAB))
print(classification_report(y_val, y_predAB, target_names=['no', 'yes']))

Train data 
 [[23336  4500]
 [ 9683 18153]]
Validation data 
 [[5856 1114]
 [ 364  547]]
              precision    recall  f1-score   support

          no       0.94      0.84      0.89      6970
         yes       0.33      0.60      0.43       911

    accuracy                           0.81      7881
   macro avg       0.64      0.72      0.66      7881
weighted avg       0.87      0.81      0.83      7881

